# Wikipedia Retriver 

In [27]:
from langchain_community.retrievers import WikipediaRetriever

In [28]:
# Initialize the retriever 
retriever = WikipediaRetriever(top_k_results=2,lang="en")

In [29]:
# Define your query
query = "Top 10 most liked anime in 2025."

# Get relevant Wikipedia documents
docs = retriever.invoke(query)

In [30]:
# Print retrived content 
for i ,doc in enumerate(docs):
    print(f"\n-- Result {i+1} --\n")
    print(f"Content : \n {doc.page_content}....")



-- Result 1 --

Content : 
 Umamusume: Pretty Derby is a Japanese multimedia franchise created by Cygames. The franchise centers on anthropomorphised racehorses known as Umamusume (ウマ娘; lit. 'horse girls'), who compete in races inspired by real-life race tracks managed by the Japan Racing Association. Most characters are named after and modeled on actual racehorses, reflecting their namesakes' personalities, racing records, and relationships with other racehorses. Initially announced in 2016 as a mobile game, the project soon expanded to include multiple anime series, manga, and live concerts. 
The game was scheduled to debut in late 2018 for iOS and Android devices, but it was delayed to a February 2021 release, with a Windows client releasing a month later on DMM Games, then later on Google Play Games in April 2023. It would later be available in Korean by Kakao Games and Traditional Chinese by Komoe Game in June 2022, with the former language getting a PC version in a stand-alone c

# Vector Store Retriever

In [31]:
# Imports 
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document
import os 
from dotenv import load_dotenv

In [32]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [33]:
# Step 2: Initialize embedding model 
model_name = os.environ.get("GOOGLE_EMBEDDING_MODEL")

embed_model = GoogleGenerativeAIEmbeddings(model = model_name)

# Step 3: Create Chroma vector store in memory
vectorStore = Chroma.from_documents(
    documents = documents, 
    embedding = embed_model,
    collection_name = "my_collection"
)


In [34]:
# Step 4: Convert vectorstore into a retriever 

retriever = vectorStore.as_retriever(search_kwargs={"k":2})

In [35]:
query = "What is chroma use for?"
results = retriever.invoke(query)

In [36]:
for i , doc in enumerate(results):
    print(f"\n-- Result {i+1} --\n")
    print(doc.page_content)


-- Result 1 --

Chroma is a vector database optimized for LLM-based search.

-- Result 2 --

Chroma is a vector database optimized for LLM-based search.


# Maximum Marginal Relevance (MMR)

In [37]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [ ]:
from langchain_community.vectorstores import FAISS

# Initialize GoogleGenerativeAIEmbeddings model
embed_model = GoogleGenerativeAIEmbeddings(model=model_name)

vectorStore = FAISS.from_documents(
    documents=docs,
    embedding=embed_model
)



In [42]:
# Enable MMR in retriever 
retriever = vectorStore.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k":3,"lambda_mult":1} # K is the top results, lambda_mult relevance-diversity balance
)

In [43]:
query = "What is langchain"
results = retriever.invoke(query)

In [44]:
for i , doc in enumerate(results):
    print(f"\n-- Result {i+1} --\n")
    print(doc.page_content)


-- Result 1 --

LangChain makes it easy to work with LLMs.

-- Result 2 --

LangChain is used to build LLM based applications.

-- Result 3 --

LangChain supports Chroma, FAISS, Pinecone, and more.


# Multi-Query Retriever

In [45]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.retrievers.multi_query import MultiQueryRetriever

# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [46]:
# create FAISS vector store. 
vectorStore = FAISS.from_documents(
    documents=all_docs,
    embedding=embed_model
    )

In [50]:
# create retriever 1 -> simple retriever 
similarity_retriever = vectorStore.as_retriever(
    search_type = "similarity",
    search_kwargs={"k":5}
)


In [52]:
# create a chat model 
from langchain_google_genai import GoogleGenerativeAI
chat_model = GoogleGenerativeAI(model=os.environ.get("GOOGLE_CHAT_MODEL"))

# create retriever 2 -> we have use simple retiever but with llm
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever = vectorStore.as_retriever(search_kwargs={"k":5}),
    llm=chat_model
)

In [53]:
query = "how to improve energy levels and maintain balance?"

# Retriver results 
similarity_results = similarity_retriever.invoke(query)
multiquery_results = multiquery_retriever.invoke(query)

In [ ]:
print("\n --- Similarity Retriever Results ----\n")
for i , doc in enumerate(similarity_results):
    print(f"\n-- Result {i+1} --\n")
    print(doc.page_content)

print("="*60)

print("\n --- Multi-Query Retriever Results ----\n")
for i , doc in enumerate(multiquery_results):
    print(f"\n-- Result {i+1} --\n")
    print(doc.page_content)

# Contextual Compression Retriver

In [59]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [56]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),
    
    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [57]:
# create FAISS vector from the documents
vectorStore = FAISS.from_documents(docs,embed_model)

In [58]:
base_retriever = vectorStore.as_retriever(search_kwargs={"k":5})

In [60]:
# set up the compressor using an LLM
compressor = LLMChainExtractor.from_llm(chat_model)

In [61]:
# create the contextual compression retriever 
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [62]:
# Query the retriever
query = "What is photosynthesis"
compressed_results = compression_retriever.invoke(query)


In [63]:
for i , doc in enumerate(compressed_results):
    print(f"\n-- Result {i+1} --\n")
    print(doc.page_content)


-- Result 1 --

Photosynthesis is the process by which green plants convert sunlight into energy.

-- Result 2 --

Photosynthesis does not occur in animal cells.

-- Result 3 --

The chlorophyll in plant cells captures sunlight during photosynthesis.
